In [504]:
#Ввод данных
#КВАРТИРЫ

import pandas as pd
import random
import math
import time

start_time = time.time()

buyers = pd.DataFrame()

data = pd.read_csv("flats_moscow.csv")

data['параметр 0'] = data['price']
data['параметр 1'] = data['totsp']

data['параметр 1'] = round(data['параметр 1'] * 1.352)

buyers.insert(0, "параметр 0", data['параметр 0'])
buyers.insert(1, "параметр 1", data['параметр 1'])

generation = buyers.copy()
counter_number = buyers.shape[1]
qual_amount = buyers.shape[1] - 1

In [505]:
#Подсчёт количества покупателей
def amount_of_buyers(fr_name, vector):
    n = fr_name.shape[0]
    counter = 0
    for j in range (0, n):
        condition = (vector[0] <= fr_name['параметр 0'][j])
        for i in range(1, qual_amount + 1):
            name = 'параметр ' + str(i)
            condition &= vector[i] >= fr_name[name][j]
        if condition:
            counter += 1
    return counter

#Подсчёт дохода одного вектора-хромосомы
def profit(fr_name, vector):
    profit = vector[0]
    for i in range(1, qual_amount + 1):
        profit -= vector[i]
    profit = profit * amount_of_buyers(fr_name, vector)
    return profit

#Подсчёт дохода всех векторов-хромосом
def profit_table(fr_name, fr1_name):
    global counter_number
    n = fr1_name.shape[0]
    profit_list = []
    for i in range (0, n):
        p = profit(fr_name, fr1_name.loc[i][:])
        profit_list.append(int(p))
    fr1_name.insert(counter_number, "прибыль", profit_list)

profit_table(buyers, generation)

In [506]:
#Формирование популяции из лучших хромосом
def generation_form(generation):
    generation = generation.sort_values(by='прибыль', ascending = False)[:20]
    generation = generation.reset_index(drop=True)
    
generation_form(generation)

In [507]:
#Наилучшие представители популяции (с наибольшей прибылью) - родители
def best_parents(fr_name):
    n = fr_name.shape[0]
    parent1 = 0
    parent2 = 0
    parent1_ind = 0
    parent2_ind = 0
    for i in range (0, n):
        if (fr_name['прибыль'][i] >= parent1):
            parent1 = fr_name['прибыль'][i]
            parent1_ind = i
        elif (fr_name['прибыль'][i] >= parent2):
            parent2 = fr_name['прибыль'][i]
            parent2_ind = i
    return parent1_ind, parent2_ind

In [508]:
#Наихудшие представители популяции (с наименьшей прибылью)- те, кто будет заменен 
def worst_parents(fr_name):
    n = fr_name.shape[0]
    w_parent1_ind = 0
    w_parent2_ind = 0
    w_parent1 = fr_name['прибыль'][0]
    w_parent2 = fr_name['прибыль'][0]
    for i in range (0, n):
        if (fr_name['прибыль'][i] <= w_parent1):
            w_parent1 = fr_name['прибыль'][i]
            w_parent1_ind = i
    for j in range (0, n):
        if (fr_name['прибыль'][j] >= w_parent1) & (fr_name['прибыль'][j] < w_parent2) & (j != w_parent1_ind):
            w_parent2 = fr_name['прибыль'][j]
            w_parent2_ind = j
    return  w_parent1_ind, w_parent2_ind, w_parent1, w_parent2

In [509]:
#Обмен качествами (одноточеченый кроссовер, точка обмена выбрана случайно)
def crossing(fr_name, ind1, ind2):
    parent1 = fr_name.loc[ind1][1:qual_amount + 1]
    parent2 = fr_name.loc[ind2][1:qual_amount + 1]
    cross_point = random.randint(1, qual_amount) 
    child1 = [fr_name.loc[ind1][0]]
    child2 = [fr_name.loc[ind2][0]]
    for i in range (0, cross_point):
        child2.append(parent1[i]) 
        child1.append(parent2[i])
    for i in range (cross_point, qual_amount):
        child1.append(parent1[i])
        child2.append(parent2[i])
    return child1, child2

In [510]:
#Мутация - изменение качеств детей
def mutation_of_qual(fr_name, vector1, vector2):
    probability = random.randint(1, 100)
    if probability <= 50:
        which_Qual = random.randint(1, qual_amount)
        which_Child = random.randint(1, 2)
        name1 = 'параметр ' + str(which_Qual)
        if which_Child == 1:
            vector1[which_Qual] = random.choice(fr_name[name1][:])
        else:
            vector2[which_Qual] = random.choice(fr_name[name1][:])
    return vector1, vector2

In [511]:
#Мутация - изменение цен для тех, кто не улучшился
def price_change(vector, fr_name):
    new = [0]
    low_border = 0
    for i in range(1, qual_amount + 1):
        low_border += vector[i]
    while (new[0] < low_border):
        new = new[0:0]
        new = [random.choice(fr_name['параметр 0'])] 
    for i in range (1, qual_amount + 1):
        new.append(vector[i])
    return new
    
def mutation(fr_name, vector1, vector2):
    vector1_profit = profit(fr_name, vector1)
    vector2_profit = profit(fr_name, vector2)
    min_profit = max(w_parent1, w_parent2)
    if vector1_profit < min_profit:
        vector1 = price_change(vector1, fr_name)
    if vector2_profit < min_profit:
        vector2 = price_change(vector2, fr_name)
    return vector1, vector2

In [512]:
#Нахождение двух худших хромосом и их замена на лучших потомков - новое поколение
def new_generation(fr_name, fr1_name, vector11, vector22):
    vector1 = vector11[0 : qual_amount + 1]
    vector2 = vector22[0 : qual_amount + 1]
    vector1_profit = profit(fr_name, vector1)
    vector2_profit = profit(fr_name, vector2)
    vector1.append(vector1_profit)
    vector2.append(vector2_profit)
    for i in range (0, qual_amount + 2):
        if (vector1_profit > w_parent1) or (vector1_profit > w_parent2):
            fr1_name.loc[w_parent1_ind][i] = vector1[i]
        if (vector2_profit > w_parent1) or (vector2_profit > w_parent2):  
            fr1_name.loc[w_parent2_ind][i] = vector2[i]
    vector1 = []
    vector2 = []
    return fr1_name

In [513]:
global_counter = 0
while global_counter < 10:
    del generation['прибыль']
    profit_table(buyers, generation)
    parent1_ind, parent2_ind = best_parents(generation)
    w_parent1_ind, w_parent2_ind, w_parent1, w_parent2 = worst_parents(generation)
    child1, child2 = crossing(generation, parent1_ind, parent2_ind)
    child1, child2 = mutation_of_qual(buyers, child1, child2)
    child1, child2 = mutation(buyers, child1, child2)
    new_generation(buyers, generation, child1, child2)
    global_counter += 1

/opt/homebrew/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [514]:
answer = 0 
n = generation.shape[0]
for i in range (0, n):
    if (generation['прибыль'][i] >= answer):
        answer = generation['прибыль'][i]
        k = i
print(answer)
print(generation.loc[k][:])
print("--- %s seconds ---" % (time.time() - start_time))

9234
параметр 0     135.0
параметр 1     108.0
прибыль       9234.0
Name: 1559, dtype: float64
--- 389.3273060321808 seconds ---
